# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [3]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [38]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [7]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


In [8]:
## added 
df.columns
##['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude','artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year']
#for i,r in df.loc[:,['song_id','title','artist_id', 'year','duration']].values:
#    print(i)
#type(df.loc[0,['song_id','title','artist_id', 'year','duration']].values.tolist()[3])
#df.loc[0,['song_id','title','artist_id', 'year','duration']].values


#Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
#       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
#      dtype='object')
df.head(n=1)



,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = list(df.loc[:,['song_id','title','artist_id', 'year','duration']].values[0])
#song_data = list(df.loc[:,['title','artist_id', 'year','duration']].values[0])
song_data
#df.loc[:,['song_id','title','artist_id', 'year','duration']]

['SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
song_table_insert = ("""INSERT INTO songs(song_id,title,artist_id,year,duration) VALUES(%s,%s,%s,%s,%s)""")
#song_table_insert = ("""INSERT INTO songs(title,artist_id,year,duration) VALUES(%s,%s,%s,%s)""")
cur.execute(song_table_insert, song_data)
conn.commit()

In [11]:
## added 
df.columns
#list(df.loc[:,['artist_id','artist_location','artist_latitude','artist_longitude']].values[0])

Index(['artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_name', 'duration', 'num_songs', 'song_id', 'title', 'year'],
      dtype='object')

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = list(df.loc[:,['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values[0])
#artist_data = list(df.loc[:,['artist_name','artist_location','artist_latitude','artist_longitude']].values[0])
artist_data
#df.loc[:,['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]

['ARNF6401187FB57032',
 'Sophie B. Hawkins',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644]

In [13]:
artist_table_insert = ("""INSERT INTO artists(artist_id  , name  , location  , lattitude  , longitude) VALUES(%s,%s,%s,%s,%s)""")
#artist_table_insert = ("""INSERT INTO artists(name ,location,lattitude,longitude) VALUES(%s,%s,%s,%s)""")

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [9]:
log_files = get_files('data/log_data')

In [10]:
filepath = log_files[0]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-09-events.json'

In [11]:
df = pd.read_json(filepath,lines=True)
df.page.value_counts()

NextSong          252
Home               20
Submit Upgrade      2
Upgrade             2
Downgrade           2
Login               1
Settings            1
Help                1
About               1
Save Settings       1
Name: page, dtype: int64

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [12]:
df = df[df.page == 'NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Beastie Boys,Logged In,Harper,M,2,Barrett,161.56689,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Lighten Up,200,1541722186796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
2,Shakira,Logged In,Harper,M,3,Barrett,145.84118,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Pienso En Ti,200,1541722347796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
3,Selena,Logged In,Harper,M,4,Barrett,172.66893,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Amor Prohibido,200,1541722492796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
4,Kid Cudi Vs Crookers,Logged In,Harper,M,5,Barrett,162.97751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Day 'N' Nite,200,1541722664796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [13]:
#df.ts
#timestamp, hour, day, week of year, month, year, and weekday 
add = pd.DataFrame({
    #'time':pd.to_datetime(df.ts,unit='ms').dt.time
                        'time':pd.to_datetime(df.ts,unit='ms')
                         ,'hour':pd.to_datetime(df.ts,unit='ms').dt.hour
                        ,'day':pd.to_datetime(df.ts,unit='ms').dt.day
                        ,'weekofyear':pd.to_datetime(df.ts,unit='ms').dt.weekofyear
                       ,'month':pd.to_datetime(df.ts,unit='ms').dt.month
                       ,'year':pd.to_datetime(df.ts,unit='ms').dt.year
                        ,'weekday':pd.to_datetime(df.ts,unit='ms').dt.weekday
                       })

In [20]:
#t = 
#t.head() 

In [21]:
time_data = ()
column_labels = ()

In [14]:
time_df = add.copy()
time_df.head()
time_table_insert = ("""INSERT INTO time(start_time , hour, day, week, month, year,weekday)
VALUES(%s,%s,%s,%s,%s,%s,%s)""")

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [24]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
user_df = df.loc[:,['userId', 'firstName','lastName','gender','level' ]]
#user_df = df.loc[:,['firstName','lastName','gender','level' ]]
#df.loc[:,['userId', 'firstName','lastName','gender','level' ]]
#df.loc[:,['userId', 'firstName','lastName','gender','level' ]].nunique().values 

In [26]:
user_table_insert = (""" INSERT INTO users(user_id,first_name,last_name,gender,level) VALUES(%s,%s,%s,%s,%s) ON CONFLICT ON CONSTRAINT users_pkey 
DO NOTHING""")
#user_table_insert = (""" INSERT INTO users(first_name,last_name,gender,level) VALUES(%s,%s,%s,%s)""")

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [33]:
#song_select = (""" SELECT song_id, duration,songs.artist_id FROM songs join artists on songs.artist_id = artists.artist_id""")
df.song.head(n=2)

0    Supermassive Black Hole (Twilight Soundtrack V...
1                                           Lighten Up
Name: song, dtype: object

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [34]:
songplay_table_insert = """INSERT INTO songplays(start_time , user_id , level , song_id , artist_id , session_id , location , user_agent ) VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"""


song_select = (""" SELECT songs.song_id, artists.artist_id  FROM songs join artists on songs.artist_id = artists.artist_id where songs.title = %s and artists.name = %s and songs.duration = %s""")
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    #print(pd.to_datetime(row.ts, unit = 'ms'))
    #print(pd.to_datetime(df.ts,unit='ms').dt.time) 
    #print(pd.datetime(row.ts,unit='ms'))
    #print(pd.datetime(row.ts,unit='ms'))
    # insert songplay record
    print(results)
    
    #songplay_data = (pd.to_datetime(row.ts,unit='ms'),row.userId,row.level,songid, artistid , row.sessionId,row.location,row.userAgent)
    #cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()

InterfaceError: cursor already closed

In [30]:
conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [31]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.